# TOKENS2

Tekst kollektsiooni on olemas printout kiht ja headerite kiht

Eesmärk: eemdldada tokens kihist need tekstid, mis on printoudiga konfliktis

Selle jaoks teha event_tokens2 (pärast saab event_tokens ümber nimetada)

Vaja on itereerida üle printoudi ja tokensi layeri ja kui nad POLE konfliktis, siis lisada tokens.

In [2]:
import json
from psycopg2 import sql
from psycopg2.sql import SQL, Identifier, Literal, DEFAULT, Composed
   
from estnltk import ElementaryBaseSpan
from estnltk.layer.layer import Layer
from estnltk.taggers import MergeTagger
from estnltk.storage.postgres import layer_table_identifier
from estnltk.converters import layer_to_dict

from cda_data_cleaning.common.luigi_tasks import CDAJob
from cda_data_cleaning.fact_extraction.event_extraction.analysis_printout_extraction.development.estnltk_collection import PgCollection


config_file = '/home/anneott/Repos/cda-data-cleaning/configurations/egcut_epi_microrun.ini'
query_length_limit = 100
storage = CDAJob.create_estnltk_storage(config_file)
prefix = 'run_202103071551'

name = prefix + '_texts'
collection = storage[name]
collection.delete()


INFO:storage.py:42: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'anneott'
INFO:storage.py:58: schema: 'work', temporary: False, role: 'egcut_epi_work_create'


In [60]:
import json
from psycopg2 import sql
from psycopg2.sql import SQL, Identifier, Literal, DEFAULT, Composed
   
from estnltk import ElementaryBaseSpan
from estnltk.layer.layer import Layer
from estnltk.taggers import MergeTagger
from estnltk.storage.postgres import layer_table_identifier
from estnltk.converters import layer_to_dict

from cda_data_cleaning.common.luigi_tasks import CDAJob
from cda_data_cleaning.fact_extraction.event_extraction.analysis_printout_extraction.development.estnltk_collection import PgCollection

#conn = CDAJob.create_postgres_connection(config_file)
#cur = conn.cursor()
#cur.execute(
#            sql.SQL(
#                """
#            set role {role};
#            drop table if exists {schema}.{table};
#            """
#            ).format(
#                role=sql.Identifier('egcut_epi_work_create'),
#                schema=sql.Identifier('work'),
#                table=sql.Identifier('run_202102252019_texts__event_segments2'),
#            )
#        )
#cur.close()
#conn.close()


#testida epi_id 32813460ga
config_file = '/home/anneott/Repos/cda-data-cleaning/configurations/egcut_epi_microrun.ini'
query_length_limit = 100
storage = CDAJob.create_estnltk_storage(config_file)
prefix = 'run_202103041142'



def iterate_spans_conflicts(layer):
    '''
    Iterates over all spans in a layer and returns two consecutive spans and if they are conflicting or not.
    Based on: https://github.com/estnltk/estnltk/blob/devel_1.6/estnltk/layer_operations/conflict_resolver.py#L13
    '''
    len_layer = len(layer)
    for i, a in enumerate(layer):
        a_end = a.end
        for j in range(i + 1, len_layer):
            b = layer[j]
            if a_end <= b.start or ( is_event_span(a) and is_event_span(b)):
                conflict = False
                yield conflict, a, b
                break
            # dont break, continue looking for conflicts for this a
            conflict = True
            yield conflict, a, b
            

def is_event_span(span):
    reg_type = span.regex_type[0]
    if reg_type:
        if reg_type.startswith('printout'):
            return False
    return True
            
# TODO: add meta
def create_event_segments2_layer(storage,
                                 prefix,
                                 new_layer_name, \
                                 input_layers=["table_segments", "event_tokens"], \
                                 debug=False):
    '''
    Create a new layer (prefix + new_layer_name + '__layers') to database,
    where all the event segment spans that are conflicting with printouts are excluded.
    '''
    columns = ["id", "text_id", "data"]
    name = prefix + '_texts'

    collection = storage[name]
    #collection.create_layer_table(new_layer_name, meta=None)
    #print('Available layers texts:', collection.layers, '\n\n\n\n')

    pgc = PgCollection(name=new_layer_name, storage=storage, version='2.0')
    #from estnltk.storage.postgres.structure.v20.collectionstructure import CollectionStructure
    #_structure = CollectionStructure(collection)
        
    print('name', name, 'new_layer_name', new_layer_name)
    with pgc.buffered_layer_insert(table_identifier=layer_table_identifier(storage, name, new_layer_name),
                                  columns=columns,
                                  query_length_limit=query_length_limit) as buffered_insert:

        for id_, input_text, meta in collection.select(
            collection_meta=["epi_id", "epi_type", "schema", "table", "field", "row_id", "effective_time"],
            layers=input_layers):
                    if  id_ != 0: #and len(input_text.text) < 270 or len([(el.start, el.end) for el in input_text.table_segments.spans]) < 3:
                        continue
                        
                    if debug:
                        print('-'*50, '\n', '-'*50, '\nid', id_)
                        print('input text', input_text)
                    
                    attributes = ["grammar_symbol", "unit_type", "value", "specialty_code", "specialty", "regex_type"]
                    new_layer = Layer(new_layer_name, attributes=attributes, text_object=input_text, ambiguous=True)

                    # merge printout and event headers to one layer
                    merge_tagger = MergeTagger(input_layers=input_layers, output_layer='merged_event_printout', output_attributes=attributes)
                    merge_tagger.tag(input_text)
                    display(input_text.merged_event_printout)

                    if debug:
                        printout_spans = [(el.start, el.end) for el in input_text.table_segments.spans]
                        event_spans = [(el.start, el.end) for el in input_text.event_tokens.spans]
                        merged_spans = [(el.start, el.end) for el in input_text.merged_event_printout.spans]
                        print('Start end spans\n*event segments: {e}\n*printouts: {p}\n*merged: {m}'.format(e=event_spans, p=printout_spans, m=merged_spans))

                    # iterate over all spans and return two consecutive spans (a,b) and if those are conflicting (conflict)
                    conflicting_span_list = []
                    for conflict, a, b in iterate_spans_conflicts(input_text.merged_event_printout):
                                               
                        if debug:
                            print('\nSpans: {a}, {b} \nConflicting: {c} '.format(c=conflict,a=(a.start, a.end), b=(b.start, b.end)))

                        are_both_event = is_event_span(a) and is_event_span(b)
                        # 1) no conflicts between events and printouts, only okay when both are events
                        # 2) not a printout span 3) has never had any conflicts
        
                        print('1)', not conflict, 'or', are_both_event, '3)', are_both_event, '4)', (a.start, a.end) not in conflicting_span_list)
                        if (not conflict or are_both_event) \
                          and is_event_span \
                          and (a.start, a.end) not in conflicting_span_list:
                          
                            if debug:
                                print('Will add this span to new layer:', (a.start, a.end))
                            for annotation in a.annotations:
                                new_layer.add_annotation(a.base_span,  **annotation) 
                                
                        #"meta: epi_id", "epi_type", "schema", "table", "field", "row_id", "effective_time"
                        # save spans that have had conflicts before, so that they wouldn't be added in the future
                        # e.g. event_spans=  (448, 487), (504, 542), (559, 597), (614, 658); printout = (450, 1768)
                        # Spans: (450, 1768), (504, 542), Conflicting: True 
                        # Spans: (450, 1768), (559, 597), Conflicting: True 
                        # Spans: (450, 1768), (614, 658), Conflicting: True 
                        # FALSEPOS: Spans: (504, 542), (559, 597), Conflicting: False, Will add this span to new layer: (504, 542)
                        # FALSEPOS: Spans: (559, 597), (614, 658), Conflicting: False, Will add this span to new layer: (559, 597)
                        # if there was a conflict between event and printout
                        if conflict and not are_both_event:
                            conflicting_span_list.append((a.start, a.end))
                            conflicting_span_list.append((b.start, b.end))
                            
                            
                        prev_span_conflict = conflict
                    
                    # add last span 
                    last = input_text.merged_event_printout[-1]
                    is_event = is_event_span(last)
                    if is_event and (last.start, last.end) not in conflicting_span_list:   
                        if debug:
                                print('Will add this last span to new layer:', (last.start, last.end))
                        #new_layer.add_annotation(ElementaryBaseSpan(last.start, last.end),ambiguous=False)
                        #new_layer.add_annotation(last.annotations) #,ambiguous=False)
                        for annotation in last.annotations:
                        
                            new_layer.add_annotation(last.base_span,  **annotation) 
                             
                    # add layer to _structure table
                    #_structure.insert(new_layer, layer_type='detached', meta=None)
                    
                    if debug:
                        display(input_text.event_tokens)
                        display(new_layer)
                    # add new layer to database
                    layer_dict = layer_to_dict(new_layer)
                    layer_json = json.dumps(layer_dict, ensure_ascii=False)
                    if debug:
                        print('layer dict', layer_dict)
                        print('layer json', layer_json)
                    #values = [id_, id_, layer_json]
                    #values = list(map(Literal, values))
                    #buffered_insert(values=values)
# TODO: last element never added (b)                    
create_event_segments2_layer(storage, 
                             prefix=prefix,
                             new_layer_name="event_tokens2",
                             debug=True)

INFO:storage.py:42: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'anneott'
INFO:storage.py:58: schema: 'work', temporary: False, role: 'egcut_epi_work_create'
name run_202103041142_texts new_layer_name event_tokens2
-------------------------------------------------- 
 -------------------------------------------------- 
id 0
input text Text(text='27.02.2014 13:52 - <ANONYM id="3" type="per" morph="_H_ sg n"/>, <ANONYM id="4" type="per" morph="_H_ sg n"/> - D00676 - E330 - pulmonoloogia \nSpirograafia on korras: \nVC 83,4%, FEV1 88,1%, PEF 106%, MEF75 102%, MEF25 61%, BDT Ventoliniga negatiivne \n\xa0 \n28.02.2014 12:39 - <ANONYM id="5" type="per" morph="_H_ sg n"/>, <ANONYM id="6" type="per" morph="_H_ sg n"/> \nANALÜÜSIDE TELLIMUS nr: 525910 \n                                 \nMATERJAL: \nIP0013948358 vB 27.02.2014 14:37 (võetud: 27.02.2014 00:00) \n                                 \nVASTUSED: \nImmuunglobuliin E seerumis 54.7 (<87.0 kU/L)   \ne1 <ANONYM i

Layer(name='merged_event_printout', attributes=('grammar_symbol', 'unit_type', 'value', 'specialty_code', 'specialty', 'regex_type'), spans=SL[Span('', [{'grammar_symbol': None, 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('27.02.2014 13:52', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '27.02.2014 13:52', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="3" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="3" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="4" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="4" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('D00676', [{'grammar_symbol': 'DOCTOR_CODE', 'unit_type': None, 'value': 'D00676', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor_code'}]),
Span('E330 - pulmonoloogia \n', [{'grammar_symbol': 'SPECIALTY', 'unit_type': None, 'value': 'whatever', 'specialty_code': 'E330', 'specialty': 'pulmonoloogia ', 'regex_type': 'specialty'}]),
Span('Spirograafia on korras: \nVC 83,4%, FEV1 88,1%, PEF 106%, MEF75 102%, MEF25 61%, BDT Ventoliniga negatiivne', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('\n\xa0 \n28.02.2014 12:39', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '28.02.2014 12:39', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="5" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="5" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="6" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="6" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('ANALÜÜSIDE TELLIMUS nr: 525910 \n                                 \nMATERJAL: \nIP0013948358 vB 27.02.2014 14:37 (võetud: 27.02.2014 00:00', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('ANALÜÜSIDE TELLIMUS nr: 525910 \n                                 \nMATERJAL: \nIP0013948358 vB 27.02.2014 14:37 (võetud: 27.02.2014 00:00) \n                                 \nVASTUSED: \nImmuunglobuliin E seerumis 54.7 (<87.0 kU/L)   \ne1 <ANONYM id="7" type="per" morph="_S_ sg n"/> kassi epiteel, kõõm <0.10 (<0.35 kU/L) 0 aste  \ne5 <ANONYM id="8" type="per" morph="_S_ sg n"/> koera kõõm <0.10 (<0.35 kU/L) 0 aste  \nt2 <ANONYM id="9" type="per" morph="_S_ sg n"/> lepa õietolm <0.10 (<0.35 kU/L) 0 aste  \nt3 <ANONYM id="10" type="per" morph="_S_ sg n"/> kase õietolm <0.10 (<0.35 kU/L) 0 aste  \nt4 <ANONYM id="11" type="per" morph="_S_ sg n"/> sarapuu õietolm <0.10 (<0.35 kU/L) 0 aste  \nw6 <ANONYM id="12" type="per" morph="_S_ sg n"/> puju õietolm <0.10 (<0.35 kU/L) 0 aste  \nw206 <ANONYM id="13" type="per" morph="_S_ sg n"/> kummeli õietolm <0.10 (<0.35 kU/L) 0 aste  \nh2 <ANONYM id="14" type="per" morph="_S_ sg n"/> kodutolm <0.10 (<0.35 kU/L) 0 aste  \nex71 <ANONYM id="15" type="per" morph="_S_ sg n"/> segu: hane-, kana-, pardi- ja kalkunisuled <0.10 (<0.35 kU/L) 0 aste  \nmx2 <ANONYM id="16" type="per" morph="_S_ sg n"/> segu: P. chrysogenum, C. herbarum, A. fumigatus, C. albicans, A. alternata, S. rostrata <0.1 (<0,35 kU/L) 0 aste  \ngx1 <ANONYM id="17" type="per" morph="_S_ sg n"/> segu: kera-, aru-, raihein, timut, aasnurmikas <0.10 (<0.35 kU/L) 0 aste  \ntx4 <ANONYM id="18" type="per" morph="_S_ sg n"/> segu: tamm, jalakas, vahtralehine plaatan, paju, pappel 0.19 (<0.35 kU/L) 0 aste  \nwx7 <ANONYM id="19" type="per" morph="_S_ sg

Start end spans
*event segments: [(0, 16), (19, 63), (65, 109), (112, 118), (121, 143), (143, 249), (250, 270), (273, 317), (319, 363), (365, 500), (537, 548), (548, 598), (599, 643), (644, 649), (650, 663), (665, 694), (695, 739), (740, 781), (782, 826), (827, 870), (871, 916), (917, 960), (961, 1006), (1007, 1053), (1054, 1099), (1100, 1145), (1146, 1191), (1192, 1238), (1239, 1284), (1285, 1326), (1327, 1372), (1373, 1384), (1386, 1399), (1400, 1447), (1448, 1493), (1494, 1501), (1503, 1517), (1519, 1530), (1532, 1544), (1546, 1557), (1559, 1571), (1573, 1612), (1613, 1658), (1659, 1670), (1672, 1685), (1687, 1705), (1707, 1737), (1738, 1783), (1784, 1794), (1796, 1817), (1818, 1831), (1833, 1870), (1871, 1916), (1917, 1921), (1923, 1941), (1943, 1950), (1952, 1967), (1969, 1993), (1995, 2046), (2049, 2094), (2096, 2141), (2144, 2150), (2153, 2173), (2173, 2190), (2190, 2203), (2204, 2210), (2212, 2338), (2340, 2366), (2367, 2436), (2438, 2483), (2484, 2493), (2494, 2515), (2516, 25

Layer(name='event_tokens', attributes=('grammar_symbol', 'unit_type', 'value', 'specialty_code', 'specialty', 'regex_type'), spans=SL[Span('27.02.2014 13:52', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '27.02.2014 13:52', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="3" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="3" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="4" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="4" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('D00676', [{'grammar_symbol': 'DOCTOR_CODE', 'unit_type': None, 'value': 'D00676', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor_code'}]),
Span('E330 - pulmonoloogia \n', [{'grammar_symbol': 'SPECIALTY', 'unit_type': None, 'value': 'whatever', 'specialty_code': 'E330', 'specialty': 'pulmonoloogia ', 'regex_type': 'specialty'}]),
Span('Spirograafia on korras: \nVC 83,4%, FEV1 88,1%, PEF 106%, MEF75 102%, MEF25 61%, BDT Ventoliniga negatiivne', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('\n\xa0 \n28.02.2014 12:39', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '28.02.2014 12:39', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="5" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="5" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="6" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="6" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('ANALÜÜSIDE TELLIMUS nr: 525910 \n                                 \nMATERJAL: \nIP0013948358 vB 27.02.2014 14:37 (võetud: 27.02.2014 00:00', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('VASTUSED: \n', [{'grammar_symbol': 'HEADERWORD', 'unit_type': None, 'value': 'VASTUSED:', 'specialty_code': None, 'specialty': None, 'regex_type': 'header_word'}]),
Span('Immuunglobuliin E seerumis 54.7 (<87.0 kU/L)   \ne1', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('<ANONYM id="7" type="per" morph="_S_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="7" type="per" morph="_S_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('kassi', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('epiteel, kõõm', [{'grammar_symbol': 'DOCTOR', 'unit_type': None, 'value': 'epiteel, kõõm', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor'}]),
Span('0.10 (<0.35 kU/L) 0 aste  \ne5', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('<ANONYM id="8" type="per" morph="_S_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="8" type="per" morph="_S_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('koera kõõm <0.10 (<0.35 kU/L) 0 aste  \nt2', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('<ANONYM id="9" type="per" morph="_S_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="9" type="per" morph="_S_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type':

Layer(name='event_tokens2', attributes=('grammar_symbol', 'unit_type', 'value', 'specialty_code', 'specialty', 'regex_type'), spans=SL[Span('', [{'grammar_symbol': None, 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('27.02.2014 13:52', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '27.02.2014 13:52', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="3" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="3" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="4" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="4" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('D00676', [{'grammar_symbol': 'DOCTOR_CODE', 'unit_type': None, 'value': 'D00676', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor_code'}]),
Span('E330 - pulmonoloogia \n', [{'grammar_symbol': 'SPECIALTY', 'unit_type': None, 'value': 'whatever', 'specialty_code': 'E330', 'specialty': 'pulmonoloogia ', 'regex_type': 'specialty'}]),
Span('Spirograafia on korras: \nVC 83,4%, FEV1 88,1%, PEF 106%, MEF75 102%, MEF25 61%, BDT Ventoliniga negatiivne', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('\n\xa0 \n28.02.2014 12:39', [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '28.02.2014 12:39', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]),
Span('<ANONYM id="5" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="5" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="6" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="6" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="20" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="20" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('<ANONYM id="21" type="per" morph="_H_ sg n"/>', [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="21" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]),
Span('D05842', [{'grammar_symbol': 'DOCTOR_CODE', 'unit_type': None, 'value': 'D05842', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor_code'}]),
Span('E340 - radioloogia \n', [{'grammar_symbol': 'SPECIALTY', 'unit_type': None, 'value': 'whatever', 'specialty_code': 'E340', 'specialty': 'radioloogia ', 'regex_type': 'specialty'}]),
Span('Röntgenuuringud \n', [{'grammar_symbol': 'HEADERWORD', 'unit_type': None, 'value': 'Röntgenuuringud', 'specialty_code': None, 'specialty': None, 'regex_type': 'header_word'}]),
Span('RÖ rindkerest', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('PA, AP', [{'grammar_symbol': 'DOCTOR', 'unit_type': None, 'value': 'PA, AP', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor'}]),
Span('otse) \nTeostatud: 27.02.2014 15:06 \n                                 \nTellimuse vastus nr: 14-59058-1 \nDiafragmakuplid kumerad', [{'grammar_symbol': 'RANDOM_TEXT', 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]),
Span('selgepiirilised, tavalisel', [{'grammar_symbol': 'DOCTOR', 'unit_type': None, 'value': 'selgepiirilised, tavalisel', 'specialty_code': None, 'specialty': None, 'regex_type': 'doctor'}]),
Span('kõrgusel. Lateraalsiinused vabad.  Hiilused rahuldava struktuursusega', [{'grammar

layer dict {'name': 'event_tokens2', 'attributes': ('grammar_symbol', 'unit_type', 'value', 'specialty_code', 'specialty', 'regex_type'), 'parent': None, 'enveloping': None, 'ambiguous': True, 'serialisation_module': None, 'meta': {}, 'spans': [{'base_span': (0, 0), 'annotations': [{'grammar_symbol': None, 'unit_type': None, 'value': None, 'specialty_code': None, 'specialty': None, 'regex_type': None}]}, {'base_span': (0, 16), 'annotations': [{'grammar_symbol': 'DATE', 'unit_type': None, 'value': '27.02.2014 13:52', 'specialty_code': None, 'specialty': None, 'regex_type': 'date'}]}, {'base_span': (19, 63), 'annotations': [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="3" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex_type': 'anonym'}]}, {'base_span': (65, 109), 'annotations': [{'grammar_symbol': 'ANONYM', 'unit_type': None, 'value': '<ANONYM id="4" type="per" morph="_H_ sg n"/>', 'specialty_code': None, 'specialty': None, 'regex

In [13]:
len([[None], ['doctor'], [None], ['doctor'], [None], ['doctor'], ['doctor'], [None], ['doctor'], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None], ['doctor'], [None]])

25

# INITIAL SOLUTION, TOO MANY LOOPS

In [3]:
from psycopg2.sql import SQL, Identifier, Literal, DEFAULT, Composed
from typing import Sequence
from estnltk.storage.postgres import layer_table_identifier
from cda_data_cleaning.fact_extraction.event_extraction.analysis_printout_extraction.development.estnltk_collection import PgCollection
from estnltk.converters import layer_to_dict
from cda_data_cleaning.common.luigi_tasks import CDAJob
config_file = '/home/anneott/Repos/cda-data-cleaning/configurations/egcut_epi_microrun.ini'
import json


            
def is_conflict(po_start, po_end, ev_start, ev_end):
    #print('-'*10)
    
    # both start and end of events span are SMALLER than printout ones
    # OR
    # both start and end of events span are BIGGER than printout ones
    if (po_end < ev_start and po_end < ev_end) or (ev_end < po_start and ev_end < po_end):
        return False
    else:
        #print('YES')
        return True

layer_name = 'headers2'
columns = ["id", "text_id", "data"]
query_length_limit = 100

tagger_input_layers = []    
progressbar = 'notebook'
missing_layer = None
storage = CDAJob.create_estnltk_storage(config_file)
prefix = 'run_202102111444'
name = 'run_202102111444_texts'
collection = storage[str(prefix) + "_texts"]
print('Available layers texts:', collection.layers, '\n\n\n\n')

pgc = PgCollection(name=name, storage=storage, version='2.0')
data_iterator = pgc.select(layers=tagger_input_layers, progressbar=progressbar, query=MissingLayerQuery(missing_layer=missing_layer) if missing_layer is not None else None)

tagger = None
collection.create_layer_table(
    layer_name='headers2', meta=None
)

from estnltk import ElementaryBaseSpan
from estnltk.layer.layer import Layer
   
with pgc.buffered_layer_insert(table_identifier=layer_table_identifier(storage, name, layer_name),
                                                columns=columns,
                                                query_length_limit=query_length_limit) as buffered_insert:
        for id_, input_text, meta in collection.select(
                    collection_meta=["epi_id", "epi_type", "schema", "table", "field", "row_id", "effective_time"],
                    layers=["table_segments", "event_headers"]):
            print('-'*50, '\n', '-'*50, '\nid', id_)
            layer = Layer('headers2', attributes=['epi_id'], text_object=input_text)
            
            
            printout_span_list = [(el.start, el.end) for el in input_text.table_segments.spans]
            print('\nprintout span list',printout_span_list)

            for span in input_text.event_headers:
                print('-'*50)
                conflicts = []
                ev_start, ev_end = span.start, span.end
                
                for po_start, po_end in printout_span_list:
                    conflict = is_conflict(po_start, po_end, ev_start, ev_end)
                    conflicts.append(conflict)
                    print('\nis conflict?', conflict)
                    print('event span start', ev_start, ', event span end', ev_end, ', printout start', po_start, ', printout end', po_end)
                                   
                print('all conflicts', conflicts)
                # if there are no conflicts with printout, we can add keep the headers by adding it to new layer headers2
                if sum(conflicts) == 0:  
                    print('\nAdd span to headers 2', span)

                    # create layer out of spans
                    layer.add_annotation(
                                ElementaryBaseSpan(ev_start, ev_end),
                                ambiguous=False,
                    )
                else:
                    print('\nDo not add span to headers 2')
                    
            layer_dict = layer_to_dict(layer)
            layer_json = json.dumps(layer_dict, ensure_ascii=False)

            values = [id_, id_, layer_json]

            values = list(map(Literal, values))
            buffered_insert(values=values)

            print('\n', meta)
            print('-'*100)
            #if not structure_written:
            #    self._structure.insert(layer=layer, layer_type='detached', meta=meta)
            #    structure_written = True
            

INFO:storage.py:42: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'anneott'
INFO:storage.py:58: schema: 'work', temporary: False, role: 'egcut_epi_work_create'
-------------------------------------------------- 
 -------------------------------------------------- 
id 0

printout span list [(118, 298), (314, 529)]
--------------------------------------------------

is conflict? False
event span start 35 , event span end 47 , printout start 118 , printout end 298

is conflict? False
event span start 35 , event span end 47 , printout start 314 , printout end 529
all conflicts [False, False]

Add span to headers 2 EnvelopingSpan(['\n28.01.2015 '], [{'DATE': '\n28.01.2015 ', 'HEADERWORD': '', 'DOCTOR': '', 'DOCTOR_CODE': '', 'SPECIALTY': '', 'SPECIALTY_CODE': '', 'ANONYM': []}])
--------------------------------------------------

is conflict? True
event span start 116 , event span end 129 , printout start 118 , printout end 298

is conflict? False
event span star


 {'epi_id': '41742326', 'epi_type': 'a', 'schema': 'original_microset_fixed', 'table': 'procedures', 'field': 'text', 'row_id': '198', 'effective_time': None}
----------------------------------------------------------------------------------------------------
-------------------------------------------------- 
 -------------------------------------------------- 
id 6

printout span list [(36, 251)]
--------------------------------------------------

is conflict? True
event span start 35 , event span end 73 , printout start 36 , printout end 251
all conflicts [True]

Do not add span to headers 2

 {'epi_id': '43602326', 'epi_type': 'a', 'schema': 'original_microset_fixed', 'table': 'procedures', 'field': 'text', 'row_id': '200', 'effective_time': None}
----------------------------------------------------------------------------------------------------
-------------------------------------------------- 
 -------------------------------------------------- 
id 7

printout span list [(36, 


 {'epi_id': '40865707', 'epi_type': 'a', 'schema': 'original_microset_fixed', 'table': 'procedures', 'field': 'text', 'row_id': '226', 'effective_time': None}
----------------------------------------------------------------------------------------------------
-------------------------------------------------- 
 -------------------------------------------------- 
id 9

printout span list [(0, 0), (0, 2120), (5874, 5998)]
--------------------------------------------------

is conflict? False
event span start 2120 , event span end 2147 , printout start 0 , printout end 0

is conflict? True
event span start 2120 , event span end 2147 , printout start 0 , printout end 2120

is conflict? False
event span start 2120 , event span end 2147 , printout start 5874 , printout end 5998
all conflicts [False, True, False]

Do not add span to headers 2
--------------------------------------------------

is conflict? False
event span start 3569 , event span end 3586 , printout start 0 , printout end 0


# DEVELOPMENT

In [54]:
from estnltk.layer_operations import iterate_conflicting_spans, resolve_conflicts
from psycopg2.sql import SQL, Identifier, Literal, DEFAULT, Composed
from typing import Sequence
from estnltk.storage.postgres import layer_table_identifier
from cda_data_cleaning.fact_extraction.event_extraction.analysis_printout_extraction.development.estnltk_collection import PgCollection
from estnltk.converters import layer_to_dict
from cda_data_cleaning.common.luigi_tasks import CDAJob
config_file = '/home/anneott/Repos/cda-data-cleaning/configurations/egcut_epi_microrun.ini'
import json
from estnltk.taggers import MergeTagger
from collections import defaultdict
from estnltk import ElementaryBaseSpan
from estnltk.layer.layer import Layer
   

layer_name = 'headers2'
columns = ["id", "text_id", "data"]
query_length_limit = 100

tagger_input_layers = []    
progressbar = 'notebook'
missing_layer = None
storage = CDAJob.create_estnltk_storage(config_file)
prefix = 'run_202102111444'
name = 'run_202102111444_texts'
collection = storage[str(prefix) + "_texts"]
print('Available layers texts:', collection.layers, '\n\n\n\n')

pgc = PgCollection(name=name, storage=storage, version='2.0')
data_iterator = pgc.select(layers=tagger_input_layers, progressbar=progressbar, query=MissingLayerQuery(missing_layer=missing_layer) if missing_layer is not None else None)

# original from estnltk
def iterate_non_conflicting_spans(layer):
    '''
    Iterates over all spans in a layer and returns NON-conflicting spans.
    '''
    len_layer = len(layer)
    for i, a in enumerate(layer):
        #print('i', i, 'a end', a.end)
        a_end = a.end
        for j in range(i + 1, len_layer):
            b = layer[j]
            #print('j', j, 'b start', b.start)
            if a_end <= b.start:
                yield a, b
                #break
            #yield a, b
            break

def iterate_spans_conflicts(layer):
    '''
    Iterates over all spans in a layer and returns NON-conflicting spans.
    '''
    len_layer = len(layer)
    for i, a in enumerate(layer):
        #print('i', i, 'a end', a.end)
        a_end = a.end
        for j in range(i + 1, len_layer):
            b = layer[j]
            #print('j', j, 'b start', b.start)
            if a_end <= b.start:
                conflict = False
                yield conflict, a, b
                break
            conflict = True
            yield conflict, a, b
            break
            

def iterate_conflicting_spans(layer):
    """Yields all pairs `(a, b)` of spans in the layer such that
    `a.start <= b.start` and `a.end > b.start`.
    :param layer: Layer
        input layer
    :returns generator
    """
    len_layer = len(layer)
    for i, a in enumerate(layer):
        a_end = a.end
        for j in range(i + 1, len_layer):
            b = layer[j]
            if a_end <= b.start:
                conflict = False
                yield conflict, a, b
                break
            conflict = True
            yield conflict, a, b
            
            
for id_, input_text, meta in collection.select(
                    collection_meta=["epi_id", "epi_type", "schema", "table", "field", "row_id", "effective_time"],
                    layers=["table_segments", "event_segments"]):
            print('-'*50, '\n', '-'*50, '\nid', id_)
            layer = Layer('event_segments2', attributes=["epi_id", "epi_type", "schema", "table", "field", "row_id", "effective_time"], text_object=input_text)
            
            
            layer1 = input_text['event_segments']
            #print('------event_headers--------')
            #display(layer1)
            
            layer2 = input_text['table_segments']
            layer2.enveloping = None
            #print('------printout-------------')
            #display(layer2)
            
            # merge printout and event headers to one layer
            # printouts should have a higher priority
            merge_tagger = MergeTagger(input_layers=["table_segments", "event_segments"], output_layer='merged_event_printout', output_attributes=['regex_type'])
            merge_tagger.tag(input_text)
            
            #tmp_layers = {}
            #tmp_layers["table_segments"] = layer2#input_text.table_segments #self.drug_name_tagger.make_layer(text=text, layers=tmp_layers, status=status)
            #tmp_layers["event_segments"] = layer1#input_text.event_headers #self.drug_code_tagger.make_layer(text=text, layers=tmp_layers, status=status)
            #merge_tagger.make_layer(text=input_text, layers=tmp_layers, status=None)
            
            # liidan kaks layerit kokku
            # siis lahendan konfliktid ära
            # allesjäänud headerid lähevad headers 2
            
            printout_spans = [(el.start, el.end) for el in input_text.table_segments.spans]
            event_spans = [(el.start, el.end) for el in input_text.event_segments.spans]
            print('event segments', event_spans)
            print('printout', printout_spans)
            #spans = printout_spans + event_spans
            #print('spans enne', spans)
            # order them by span start
            #spans = sorted(spans, key=lambda span: span[0])
            #print('spans pärast', spans)
            #print('\nbefore resolving\n')
            #display(input_text.merged_event_printout)
            print('merged spans', [(el.start, el.end) for el in input_text.merged_event_printout.spans])
            
            # PROBLEM: with resolve_confilcts: no possibility to delete BOTH conflicting
            #resolved = resolve_conflicts(input_text.merged_event_printout, conflict_resolving_strategy = 'MIN')
            #print('\nresolved\n')
            #print(resolved.spans)
            #print([(s.start, s.end) for s in resolved.spans])
            #print('annotations:', [s.annotations for s in resolved.spans])
            #display(resolved)
            
            
            prev_span_conflict = False
            # BOTH CONFLICT AND NON CONFLICT
            for conflict, a, b in iterate_spans_conflicts(input_text.merged_event_printout):
                print('\nConflicting:\n', conflict)
                print((a.start, a.end), (b.start, b.end))
                
                # 1) no conflicts between events and printouts 2) not a printout span
                if not conflict and not prev_span_conflict and a.regex_type[0] is None:
                    print('WILL ADD TO HEADERS2', (a.start, a.end))
                prev_span_conflict = conflict
            
            # PROBLEM WITH iterate_conflicting_spans, returns only conflicting spans,
            # PROBLEM: when checing prev, then here (118, 298), (129, 312), (314, 529), 129, 312 will be non conflicg
            # but we need to access NON conflicting spans
            #prev_span_conflict = False
            #for conflict, a, b in iterate_conflicting_spans(input_text.merged_event_printout):
            #    print('\nConflicting:\n', conflict)
            #    print((a.start, a.end), (b.start, b.end))
            #    if not conflict and not prev_span_conflict:
            #        print('WILL ADD TO HEADERS2', (a.start, a.end))
            #    prev_span_conflict = conflict
                        
            #number_of_conflicts = 0
            #map_conflicts_new = defaultdict(set)
            
            # NON CONFLICT
            # need to check that there is no conflict with neither previous or following
            #[(0, 35), (47, 116), (118, 298), (129, 312), (314, 529), (351, 547)]
            #prev_conflict = False 
            #for a, b, conflict in iterate_non_conflicting_spans(input_text.merged_event_printout):
            #    print('\nNON Conflicting:\n')
            #    print((a.start, a.end), (b.start, b.end))
            #    map_conflicts_new[a.base_span].add(b.base_span)
            #    map_conflicts_new[b.base_span].add(a.base_span)
            #    print('map_conflicts_new',map_conflicts_new)
            #    print('a', a, 'b', b)
            #    #number_of_conflicts += 1
            #    if not conflict and prev_not_conflict:
            #        print('WILL ADD TO HEADERS2', (a.start, a.end))
            #    prev_conflict = conflict
                #layer.add_annotation(
                #                ElementaryBaseSpan(ev_start, ev_end),
                #                ambiguous=False,
                #    )
            
            print('number of conflicts:', number_of_conflicts)
            

INFO:storage.py:42: connecting to host: '10.6.6.29', port: '5432', dbname: 'egcut_epi', user: 'anneott'
INFO:storage.py:58: schema: 'work', temporary: False, role: 'egcut_epi_work_create'
Available layers texts: ['anonymised', 'type1', 'type2', 'type3', 'type4', 'type5', 'type6', 'type7', 'table_segments', 'event_tokens', 'event_headers', 'event_segments'] 




-------------------------------------------------- 
 -------------------------------------------------- 
id 0
event segments [(0, 35), (47, 116), (129, 312), (351, 547)]
printout [(118, 298), (314, 529)]
merged spans [(0, 35), (47, 116), (118, 298), (129, 312), (314, 529), (351, 547)]

Conflicting:
 False
(0, 35) (47, 116)
WILL ADD TO HEADERS2 (0, 35)

Conflicting:
 False
(47, 116) (118, 298)
WILL ADD TO HEADERS2 (47, 116)

Conflicting:
 True
(118, 298) (129, 312)

Conflicting:
 False
(129, 312) (314, 529)

Conflicting:
 True
(314, 529) (351, 547)
number of conflicts: 3
-------------------------------------------------- 
 ------